# Import Packages and Load Dataset

In [4]:
import nengo
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

%matplotlib widget

from matplotlib import cm
from scipy.stats import gennorm
from IPython import display
from TacSet import TacSet


# Load dataset
streams = np.load('touch_stream.npz')
[sSharp, sRound, sPlateau] = [streams[file] for file in streams.files]
labels = ['sharp', 'round', 'plateau']

# Visualize Dataset

In [5]:
frames = np.min([len(sSharp), len(sRound), len(sPlateau)])

# Visualize data steam
fig, axs = plt.subplots(1, 3, figsize=(15, 5), dpi=100)
im0 = axs[0].imshow(sSharp[0])
im1 = axs[1].imshow(sRound[0])
im2 = axs[2].imshow(sPlateau[0])
def update(frame):
    im0.set_data(sSharp[frame])
    im1.set_data(sRound[frame])
    im2.set_data(sPlateau[frame])
    return [im0, im1, im2]
anim = animation.FuncAnimation(fig=fig, func=update, frames=frames, interval=30, blit=True, repeat=False)
video = anim.to_html5_video()
html = display.HTML(video)
display.display(html)
plt.close()  

# Generalized Gaussian Spiking Encoding for Tactile Sensors

In [7]:
tacset = TacSet([sSharp, sRound, sPlateau], labels)

(array([[ 1.41342408e-10, -1.23683917e-08,  1.66237827e-07,
         -4.59902893e-05, -4.10629269e-05, -3.53128506e-05,
         -3.58203254e-05, -3.30459043e-05, -4.88785860e-09,
         -1.86167953e-12, -4.62122913e-16,  1.43136996e-18,
          1.41139484e-18,  1.41135501e-18,  1.41171323e-18],
        [-1.12349974e-09, -5.08729916e-09, -6.45841352e-08,
         -4.94289453e-05, -4.50552024e-05, -4.02614927e-05,
         -2.95523735e-05, -3.60967037e-05,  5.51248543e-09,
          1.68256554e-13, -5.52971908e-16,  1.41693109e-18,
          1.41218699e-18,  1.41127033e-18,  1.41147380e-18],
        [ 1.76197155e-11, -2.03345011e-09, -3.14228732e-08,
         -4.42484842e-05, -3.81452811e-05, -3.38069014e-05,
         -3.15379948e-05, -2.97177434e-05, -4.53767486e-09,
          1.61706815e-12,  1.55768554e-16,  1.41267342e-18,
          1.40993560e-18,  1.41175701e-18,  1.41282314e-18],
        [-7.65424244e-12, -7.70505364e-10,  4.63647431e-09,
         -3.66631000e-05, -3.51271795